In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch_geometric
device =  torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
from tqdm.notebook import tqdm
import numpy as np
import os
import sys
import os.path as osp
import random

root_file = "/home/h8lee/DSC180B-A11-Project/src"

In [3]:
import yaml

with open(os.path.join(root_file, 'reg_defs.yml')) as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    definitions = yaml.load(file, Loader=yaml.FullLoader)
    
features = definitions['features']
spectators = definitions['spectators']
labels = definitions['labels']

nfeatures = definitions['nfeatures']
# nspectators = definitions['nspectators']
nlabels = definitions['nlabels']
# ntracks = definitions['ntracks']

In [4]:
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.nn import EdgeConv, global_mean_pool
from torch.nn import Sequential as Seq, Linear as Lin, ReLU, BatchNorm1d, Flatten, Module
from torch_scatter import scatter_mean
from torch_geometric.nn import MetaLayer

In [5]:
from torch_geometric.data import Data, DataListLoader, Batch
from torch.utils.data import random_split
import os.path as osp
from GraphDataset import GraphDataset
from load_data import path_generator
import glob
import shutil

In [7]:
train_files = random.sample(path_generator('signal', eda=False), 12)
dir_path = '/home/h8lee/DSC180B-A11-Project'

In [8]:
train_files

['/home/h8lee/teams/DSC180A_FA21_A00/a11/train_mass_hbb/BulkGravitonToHHTo4Q_MX-600to6000_MH-15to250_part1_TuneCP5_13TeV-madgraph_pythia8/nano_mc2017_130_Skim.root',
 '/home/h8lee/teams/DSC180A_FA21_A00/a11/train_mass_hbb/BulkGravitonToHHTo4Q_MX-600to6000_MH-15to250_part1_TuneCP5_13TeV-madgraph_pythia8/nano_mc2017_2-67_Skim.root',
 '/home/h8lee/teams/DSC180A_FA21_A00/a11/train_mass_hbb/BulkGravitonToHHTo4Q_MX-600to6000_MH-15to250_part1_TuneCP5_13TeV-madgraph_pythia8/nano_mc2017_1-11_Skim.root',
 '/home/h8lee/teams/DSC180A_FA21_A00/a11/train_mass_hbb/BulkGravitonToHHTo4Q_MX-600to6000_MH-15to250_part1_TuneCP5_13TeV-madgraph_pythia8/nano_mc2017_2-129_Skim.root',
 '/home/h8lee/teams/DSC180A_FA21_A00/a11/train_mass_hbb/BulkGravitonToHHTo4Q_MX-600to6000_MH-15to250_part1_TuneCP5_13TeV-madgraph_pythia8/nano_mc2017_61_Skim.root',
 '/home/h8lee/teams/DSC180A_FA21_A00/a11/train_mass_hbb/BulkGravitonToHHTo4Q_MX-600to6000_MH-15to250_part1_TuneCP5_13TeV-madgraph_pythia8/nano_mc2017_1-142_Skim.root',

In [9]:
# from RegDataGenerator import DataGenerator
# train_files = ['/teams/DSC180A_FA21_A00/a11/train_mass_hbb/BulkGravitonToHHTo4Q_MX-600to6000_MH-15to250_part1_TuneCP5_13TeV-madgraph_pythia8/nano_mc2017_1-1_Skim.root']
# dir_path = '/home/h8lee/DSC180B-A11-Project/'

# Optimize runtime
graph_dataset = GraphDataset(dir_path, features, labels, spectators, n_events=1000, n_events_merge=1, 
                             file_names= train_files)

Processing...


  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

...

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

Done!


In [12]:
def collate(items):
    l = sum(items, [])
    return Batch.from_data_list(l)

torch.manual_seed(0)
valid_frac = 0.20
full_length = len(graph_dataset)
valid_num = int(valid_frac*full_length)
batch_size = 32

train_dataset, valid_dataset = random_split(graph_dataset, [full_length-valid_num,valid_num])

train_loader = DataListLoader(train_dataset, batch_size=batch_size, pin_memory=True, shuffle=True)
train_loader.collate_fn = collate
valid_loader = DataListLoader(valid_dataset, batch_size=batch_size, pin_memory=True, shuffle=False)
valid_loader.collate_fn = collate

train_samples = len(train_dataset)
valid_samples = len(valid_dataset)

/opt/conda/lib/python3.9/site-packages/torch_geometric/deprecation.py:13: UserWarning: 'data.DataListLoader' is deprecated, use 'loader.DataListLoader' instead
  warnings.warn(out)


In [13]:
# Fixable

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layer = Seq(Lin(30, 32),
                         Lin(32, 64),
                         BatchNorm1d(64),
                         Lin(64, 32),
                         BatchNorm1d(32),
                         Lin(32, 32),
                         BatchNorm1d(32),
                         ReLU(),
                         Lin(32,1))

    def forward(self, x, batch):
        out = scatter_mean(x, batch, dim=0)
        return self.layer(out)
    
# net = Net().to(device)
# optimizer = torch.optim.Adam(net.parameters(), lr=0.02)

In [14]:
n_epochs = 8
t = tqdm(range(0, n_epochs))

  0%|          | 0/8 [00:00<?, ?it/s]

In [15]:
lst = []
best_loss = float(np.inf)
net = Net().to(device) # Model initialization
optimizer = torch.optim.Adam(net.parameters(), lr=0.02)

for epoch in t:
    p = tqdm(enumerate(train_loader), total=train_samples/batch_size, leave=bool(epoch==n_epochs-1)) # Are we only
                                                                                                # using training set
    loss_func = nn.MSELoss()
    if (epoch > 0) & (os.path.exists('simplenetwork_best.pt')):
        # We restore the weights configured for the model from first epoch
        net.load_state_dict(torch.load('simplenetwork_best.pt'))

    temp = []
    net.train()
    for i, data in p:
        data = data.to(device) # Moving data to memory
        y = data.y # Retrieving target variable
        prediction = net(data.x, data.batch) # `data` being used here should be validation set
        loss = loss_func(prediction.float(), y.float()) # Will this be training loss or validation loss
        temp.append(loss.item())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step() # Updating optimizer
        
    batch_loss = np.average(temp)
    lst.append(batch_loss)
    if batch_loss < best_loss:
        best_loss = batch_loss
        modpath = osp.join('simplenetwork_best.pt')
        print('New best model saved to:',modpath)
        torch.save(net.state_dict(),modpath)

  0%|          | 0/25.0 [00:00<?, ?it/s]

New best model saved to: simplenetwork_best.pt


  0%|          | 0/25.0 [00:00<?, ?it/s]

New best model saved to: simplenetwork_best.pt


  0%|          | 0/25.0 [00:00<?, ?it/s]

...

New best model saved to: simplenetwork_best.pt


  0%|          | 0/25.0 [00:00<?, ?it/s]

New best model saved to: simplenetwork_best.pt


  0%|          | 0/25.0 [00:00<?, ?it/s]

New best model saved to: simplenetwork_best.pt


In [16]:
lst

[32080.440244140624,
 28007.71109375,
 24508.260390625,
 21787.5181640625,
 21549.60525390625,
 21474.535,
 21195.399296875,
 20758.62728515625]

In [12]:
lst

[11585.3248828125,
 8948.7143359375,
 6875.442177734375,
 6342.92619140625,
 6123.60416015625,
 6060.99939453125,
 6036.7575,
 6005.0105859375]

In [13]:
[np.sqrt(x) for x in lst]

[107.63514706085786,
 94.5976444523726,
 82.91828614807699,
 79.64248986192138,
 78.25346101071985,
 77.8524206080405,
 77.69657328351103,
 77.49200336768627]

In [ ]:
lst

----

In [ ]:
def test(net, data_loader, data_samples, batch_size, n_epochs):
    net.eval()
    t = tqdm(enumerate(data_loader), total=data_samples/batch_size, leave=bool(epoch==n_epochs-1))
    loss_func = nn.L1Loss()
    #track total loss, track batch mean, track number of batches
    data_mean = 0
    total_loss = 0
    batches = 0
    #
    for i, data in t:
        data = data.to(device)
        y = data.y
        prediction = net(data.x, data.batch)
        loss = loss_func(prediction.float(), y.float())
        # add batch mean to total, add loss
        data_mean += torch.mean(data.x)
        total_loss += loss
        batches += 1
        #
        t.refresh()
    return total_loss/(data_mean/batches)

## RUN THE WEIGHTED MODEL ON TEST SET

In [ ]:
# Early stopper: When validation loss stops decreasing

In [ ]:
inputs = 19
hidden = 40
outputs = 1

class Edge(torch.nn.Module):
    def __init__:
    def forward():
        
class NodeBlock(torch.nn.Module):
    def __init__:
    def forward():
        
class GlobalBlock(torch.nn.Module):
    def __init__:
    def forward():
        
class InteractionNetwork(torch.nn.Module):
    def __init__:
    def forward():
        

In [ ]:
test_files = ["/home/h8lee/teams/DSC180A_FA21_A00/a11/test_mass_hbb/BulkGravitonToHHTo4Q_MX-600to6000_MH-15to250_part1_TuneCP5_13TeV-madgraph_pythia8/nano_mc2017_3-10_Skim.root"]
#delete old processed files
dir_path = '/home/h8lee/DSC180B-A11-Project/test/'
try:
    shutil.rmtree(dir_path)
except OSError as e:
    print("Error: %s : %s" % (dir_path, e.strerror))
test_dataset = GraphDataset(dir_path, features, labels, spectators, n_events=2000, n_events_merge=1, 
                            file_names=test_files)

test_loader = DataListLoader(test_dataset, batch_size=batch_size, pin_memory=True, shuffle=False)
test_loader.collate_fn = collate
test_samples = len(test_dataset)

In [ ]:
y_test = []
y_predict = []
t = tqdm(enumerate(test_loader), total=train_samples/batch_size, leave=bool(epoch==n_epochs-1))
for i,data in t:
    data = data.to(device)    
    batch_output = net(data.x, data.batch)    
    y_predict.append(batch_output.detach().cpu().numpy())
    y_test.append(data.y.cpu().numpy())
y_test = np.concatenate(y_test)
y_predict = np.concatenate(y_predict)

In [ ]:
#rmse
np.sqrt(np.mean((y_test - y_predict) ** 2))